# Excel 読み込みサンプル

リポジトリ実装 (`ExcelLogRepository`) とユースケース (`LoadLogsUseCase`) を使い、data 配下の Excel を読み込みます。

In [ ]:
import pandas as pd
import datetime as dt
from pathlib import Path

from src.application.use_cases import LoadLogsUseCase
from src.infrastructure.excel_repository import ExcelLogRepository

pd.set_option("display.max_columns", 40)

excel_path = Path("data/★機器遠隔操作履歴＆機器状態変化履歴_1016 - コピー.xlsx")
excel_path.exists()

## 1. 対象契約IDと期間のあたりを付ける
フルデータは重いので、先に契約IDと日付の最小値だけを読む。

In [ ]:
op_cols_light = ["ContractId", "OrderReceiptDate"]
op_preview = pd.read_excel(
    excel_path,
    sheet_name="機器遠隔操作履歴",
    usecols=op_cols_light,
    engine="openpyxl",
)
contract_ids = op_preview["ContractId"].dropna().unique().tolist()[:5]  # 必要に応じて調整
start_date = pd.to_datetime(op_preview["OrderReceiptDate"], errors="coerce").min()

contract_ids, start_date

## 2. ユースケース経由で取得
必要に応じて `contract_ids` や `start_date` を書き換えてください。

In [ ]:
repo = ExcelLogRepository()
use_case = LoadLogsUseCase(repo)

merged = use_case.execute(
    contract_ids=contract_ids,
    start_date=start_date.to_pydatetime() if hasattr(start_date, "to_pydatetime") else start_date,
    end_date=None,  # 期間上限を付けたい場合は datetime を指定
)
merged.df.head(), len(merged.df)